In [2]:
from mip import *
import datetime
import pandas as pd
import numpy as np
from EMS import *

## DayAhead PARAMETERS

In [3]:
PARAM_DayAhead = {}
# add length check with res & horizon
PARAM_DayAhead['Horizon'] = 3*24*60        # horizon to optimize (min)
PARAM_DayAhead['Resolution'] = 15    # sampling period(min)
PARAM_DayAhead['PV_capacity'] = 50   # (kw) PV sizing for this EMS
TOU = getBuySellrate(Resolution=PARAM_DayAhead['Resolution'],
                                    Horizon=PARAM_DayAhead['Horizon'],
                                    TOU_CHOICE='THcurrent',
                                    start_time=datetime.timedelta(minutes=0))
PARAM_DayAhead['Buy_rate'] = TOU['buy'].to_numpy()
PARAM_DayAhead['Sell_rate'] = TOU['sell'].to_numpy()

PARAM_DayAhead['battery'] = {}
PARAM_DayAhead['battery']['charge_effiency'] = [0.95,0.95];              #  bes charge eff
PARAM_DayAhead['battery']['discharge_effiency'] = [0.95*0.93,0.95*0.93]; #  bes discharge eff note inverter eff 0.93-0.96
PARAM_DayAhead['battery']['discharge_rate'] = [30,30]; # kW max discharge rate
PARAM_DayAhead['battery']['charge_rate'] = [30,30]; # kW max charge rate
PARAM_DayAhead['battery']['actual_capacity'] = [125,125]; # kWh soc_capacity 
PARAM_DayAhead['battery']['initial'] = [50,50]; # userdefined int 0-100 %
PARAM_DayAhead['battery']['min'] = [20,20]; #min soc userdefined int 0-100 %
PARAM_DayAhead['battery']['max'] = [80,80]; #max soc userdefined int 0-100 %
PARAM_DayAhead['battery']['num_batt'] = len(PARAM_DayAhead['battery']['actual_capacity'])

## Intraday PARAMETERS

In [4]:
PARAM_Intraday = {}
# add length check with res & horizon
PARAM_Intraday['Horizon'] = 1*60        # horizon to optimize (min)
PARAM_Intraday['Resolution'] = 5    # sampling period(min)
PARAM_Intraday['PV_capacity'] = 50   # (kw) PV sizing for this EMS
TOU = getBuySellrate(Resolution=PARAM_Intraday['Resolution'],
                                    Horizon=PARAM_Intraday['Horizon'],
                                    TOU_CHOICE='THcurrent',
                                    start_time=datetime.timedelta(minutes=0))
PARAM_Intraday['Buy_rate'] = TOU['buy'].to_numpy()
PARAM_Intraday['Sell_rate'] = TOU['sell'].to_numpy()
PARAM_Intraday['battery'] = {}
PARAM_Intraday['battery']['charge_effiency'] = [0.95,0.95];              #  bes charge eff
PARAM_Intraday['battery']['discharge_effiency'] = [0.95*0.93,0.95*0.93]; #  bes discharge eff note inverter eff 0.93-0.96
PARAM_Intraday['battery']['discharge_rate'] = [30,30]; # kW max discharge rate
PARAM_Intraday['battery']['charge_rate'] = [30,30]; # kW max charge rate
PARAM_Intraday['battery']['actual_capacity'] = [125,125]; # kWh soc_capacity 
PARAM_Intraday['battery']['initial'] = [50,50]; # userdefined int 0-100 %
PARAM_Intraday['battery']['min'] = [20,20]; #min soc userdefined int 0-100 %
PARAM_Intraday['battery']['max'] = [80,80]; #max soc userdefined int 0-100 %
PARAM_Intraday['battery']['num_batt'] = len(PARAM_Intraday['battery']['actual_capacity'])

In [5]:
root_folder = 'C:/Users/User/Desktop/VSCpython/EMS_on_production/input_data/predict/'
load_dayAhead = pd.read_csv(root_folder + 'load_Dayahead_20230302_20231228.csv',parse_dates=['ds'])
pv_dayahead = pd.read_csv(root_folder + 'pv_Dayahead_20230102_20231228.csv',parse_dates=['ds'])
load_intraday = pd.read_csv(root_folder + 'load_Intraday_20230301_20231230.csv',parse_dates=['ds'])
pv_intraday = pd.read_csv(root_folder + 'pv_Intraday_20230101_20231231.csv',parse_dates=['ds'])
pv_scaling_factor = 50/8 # scale PV from 8 kW to 50 kW conversion factor conversion kW/(W/m^2)
# scale PV and PV >= 0
pv_dayahead.iloc[:,1:] = pv_dayahead.iloc[:,1:].clip(lower=0)*pv_scaling_factor
pv_intraday.iloc[:,1:] = pv_intraday.iloc[:,1:].clip(lower=0)*pv_scaling_factor
# load must >= 0
load_dayAhead.iloc[:,1:] = load_dayAhead.iloc[:,1:].clip(lower=0)
load_intraday.iloc[:,1:] = load_intraday.iloc[:,1:].clip(lower=0)




## Rolling

In [6]:
Num_days = 4
initial_date = pd.to_datetime('2023-11-01')
initial_time = pd.to_datetime('2023-11-01')
dayAhead_sol_list = pd.DataFrame()
IntraDay_sol_list = pd.DataFrame()
Resolution_Intraday_HR = PARAM_Intraday['Resolution']/60
Resolution_dayAhead_HR = PARAM_DayAhead['Resolution']/60
First_time_init = True # boolean for checking if it is first day of rolling
for day in range(Num_days) :
    PARAM_DayAhead['Start_date'] = initial_date + pd.Timedelta(days=day)
    PARAM_DayAhead['PV'] = pv_dayahead[pv_dayahead['ds'] == PARAM_DayAhead['Start_date'] ].iloc[:,1:].to_numpy().flatten()
    PARAM_DayAhead['PL'] = load_dayAhead[load_dayAhead['ds'] == PARAM_DayAhead['Start_date']].iloc[:,1:].to_numpy().flatten()    
    # set initial soc
    if day == 0:
        PARAM_DayAhead['battery']['initial'] = [50,50]
    else :
                       
        PARAM_DayAhead['battery']['initial'] = [dayAhead_sol.iloc[96,:][f'soc_{j}'] for j in range(PARAM_DayAhead['battery']['num_batt'])]
        
    #get DA sol
    print(f'begin optimizing day{day}')
    dayAhead_sol = EMS_econ_opt(PARAM_DayAhead,energycost=1,multibatt=1,chargebatt=1,smoothcharge=0.3)
    dayAhead_sol_list = pd.concat([dayAhead_sol_list,dayAhead_sol.iloc[:96,:]])
    print(f'finish optimizing day{day}')
    # rolling 5 minute 
    for i in range(288) :
        PARAM_Intraday['Start_date'] = PARAM_DayAhead['Start_date'] + pd.Timedelta(minutes=5*i)
        PARAM_Intraday['End_date'] = PARAM_Intraday['Start_date'] + pd.Timedelta(minutes=PARAM_Intraday['Horizon'])
        PARAM_Intraday['PV'] = pv_intraday[pv_intraday['ds'] == PARAM_Intraday['Start_date'] ].iloc[:,1:].to_numpy().flatten()
        PARAM_Intraday['PL'] = load_intraday[load_intraday['ds'] == PARAM_Intraday['Start_date'] ].iloc[:,1:].to_numpy().flatten()
        windowed_dayAhead_plan = dayAhead_sol[ (dayAhead_sol['datetime'] >= PARAM_Intraday['Start_date'] - pd.Timedelta(minutes=10)) 
                                              & (dayAhead_sol['datetime'] < PARAM_Intraday['End_date'])]
        # use DA solution as parameters for HA
       
        PARAM_Intraday['Pchg'] = windowed_dayAhead_plan['Pchg_0'].to_numpy().flatten()
        PARAM_Intraday['Pdchg'] = windowed_dayAhead_plan['Pdchg_0'].to_numpy().flatten()
        PARAM_Intraday['Pnet'] = windowed_dayAhead_plan['Pnet'].to_numpy().flatten()
        TOU = getBuySellrate(Resolution=PARAM_Intraday['Resolution'],
                                    Horizon=PARAM_Intraday['Horizon'],
                                    TOU_CHOICE='THcurrent',
                                    start_time=datetime.timedelta(minutes=5*i))
        PARAM_Intraday['Buy_rate'] = TOU['buy'].to_numpy()
        PARAM_Intraday['Sell_rate'] = TOU['sell'].to_numpy()
        #set initial soc
        if First_time_init:
            PARAM_Intraday['battery']['initial'] = [50,50]
            First_time_init = False
        else :
                       
            PARAM_Intraday['battery']['initial'] = [intraday_sol.iloc[1,:][f'soc_{j}'] for j in range(PARAM_Intraday['battery']['num_batt'])]
        print(f'begin optimizing day {day} step {i}')
        # get HA sol
        intraday_sol = EMS_rolling(PARAM_Intraday,minute=5*i,energycost=1,Pnet_diff=0.1,Pchg_diff=0.3,Pdchg_diff=0.2,chargebatt=0.07,smoothcharge=0.06)
        # if i < 276 :
        #     IntraDay_sol_list = pd.concat([IntraDay_sol_list,intraday_sol.iloc[0:1,:]])
        # else :
        #     IntraDay_sol_list = pd.concat([IntraDay_sol_list,intraday_sol])
        IntraDay_sol_list = pd.concat([IntraDay_sol_list,intraday_sol.iloc[0:1,:]])
        print(f'finish optimizing day {day} step {i}')


    
    
    
    

begin optimizing day0
finish optimizing day0
begin optimizing day 0 step 0
finish optimizing day 0 step 0
begin optimizing day 0 step 1
finish optimizing day 0 step 1
begin optimizing day 0 step 2
finish optimizing day 0 step 2
begin optimizing day 0 step 3
finish optimizing day 0 step 3
begin optimizing day 0 step 4
finish optimizing day 0 step 4
begin optimizing day 0 step 5
finish optimizing day 0 step 5
begin optimizing day 0 step 6
finish optimizing day 0 step 6
begin optimizing day 0 step 7
finish optimizing day 0 step 7
begin optimizing day 0 step 8
finish optimizing day 0 step 8
begin optimizing day 0 step 9
finish optimizing day 0 step 9
begin optimizing day 0 step 10
finish optimizing day 0 step 10
begin optimizing day 0 step 11
finish optimizing day 0 step 11
begin optimizing day 0 step 12
finish optimizing day 0 step 12
begin optimizing day 0 step 13
finish optimizing day 0 step 13
begin optimizing day 0 step 14
finish optimizing day 0 step 14
begin optimizing day 0 step 15

In [7]:
dayAhead_sol

,datetime,PARAM_PV,PARAM_PL,Buy_rate,Sell_rate,Pnet,u1,s1,Pchg_0,Pdchg_0,...,soc_0,upper_bound_Pchg_0,upper_bound_Pdchg_0,Pchg_1,Pdchg_1,xchg_1,xdchg_1,soc_1,upper_bound_Pchg_1,upper_bound_Pdchg_1
0,2023-11-04 00:00:00,0.000000,0.464476,2.6,2,-3.122737,2.029779,0.0,1.329131,0.000000,...,56.041817,0.0,0.0,1.329131,0.000000,1.0,0.0,56.041817,0.0,0.0
1,2023-11-04 00:15:00,0.000000,0.442132,2.6,2,-3.100393,2.015256,0.0,1.329131,0.000000,...,56.294352,0.0,0.0,1.329131,0.000000,1.0,0.0,56.294352,0.0,0.0
2,2023-11-04 00:30:00,0.000000,0.360393,2.6,2,-3.018654,1.962125,0.0,1.329131,0.000000,...,56.546887,0.0,0.0,1.329131,0.000000,1.0,0.0,56.546887,0.0,0.0
3,2023-11-04 00:45:00,0.000000,0.329606,2.6,2,-2.987867,1.942114,0.0,1.329131,0.000000,...,56.799421,0.0,0.0,1.329131,0.000000,1.0,0.0,56.799421,0.0,0.0
4,2023-11-04 01:00:00,0.000000,0.355300,2.6,2,-3.013561,1.958815,0.0,1.329131,0.000000,...,57.051956,0.0,0.0,1.329131,0.000000,1.0,0.0,57.051956,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2023-11-06 22:45:00,0.204107,3.886803,5.8,2,1.219982,0.000000,0.0,0.000000,2.451339,...,48.950393,0.0,0.0,0.000000,2.451339,0.0,1.0,48.950393,0.0,0.0
284,2023-11-06 23:00:00,0.000000,3.540656,5.8,2,1.362021,0.000000,0.0,0.000000,2.451339,...,48.395478,0.0,0.0,0.000000,2.451339,0.0,1.0,48.395478,0.0,0.0
285,2023-11-06 23:15:00,0.010483,3.609791,5.8,2,1.303369,0.000000,0.0,0.000000,2.451339,...,47.840562,0.0,0.0,0.000000,2.451339,0.0,1.0,47.840562,0.0,0.0
286,2023-11-06 23:30:00,0.000000,3.420045,5.8,2,1.482632,0.000000,0.0,0.000000,2.451339,...,47.285647,0.0,0.0,0.000000,2.451339,0.0,1.0,47.285647,0.0,0.0


In [8]:
intraday_sol

,datetime,PARAM_PV,PARAM_PL,Buy_rate,Sell_rate,Pnet,u1,s1,Pchg_0,Pdchg_0,...,Pchg_1,Pdchg_1,xchg_1,xdchg_1,soc_1,upper_bound_Pchg_1,upper_bound_Pdchg_1,uPnet,uPchg,uPdchg
0,2023-11-04 23:55:00,0.015710,1.586237,5.8,2,1.110223e-16,0.0,0.000000,0.0,0.785264,...,0.0,0.785264,0.0,1.0,23.571238,0.0,0.027841,0.000000,0.000000,0.785264
1,2023-11-05 00:00:00,0.005855,1.632063,2.6,2,-1.110223e-16,0.0,0.000000,0.0,0.813104,...,0.0,0.813104,0.0,1.0,23.511984,0.0,0.091018,0.000000,0.000000,0.813104
2,2023-11-05 00:05:00,0.000000,1.444172,2.6,2,1.110223e-16,0.0,0.000000,0.0,0.722086,...,0.0,0.722086,0.0,1.0,23.450629,0.0,0.003675,0.000000,0.000000,0.722086
3,2023-11-05 00:10:00,0.000000,1.451522,2.6,2,-1.110223e-16,0.0,0.000000,0.0,0.725761,...,0.0,0.725761,0.0,1.0,23.396142,0.0,0.036170,0.000000,0.000000,0.725761
4,2023-11-05 00:15:00,0.000000,1.379181,2.6,2,-1.110223e-16,0.0,0.000000,0.0,0.689591,...,0.0,0.689591,0.0,1.0,23.341378,0.0,0.021572,0.000000,0.000000,0.689591
5,2023-11-05 00:20:00,0.000000,1.336037,2.6,2,0.000000e+00,0.0,0.000000,0.0,0.668018,...,0.0,0.668018,0.0,1.0,23.289343,0.0,0.000970,0.000000,0.000000,0.668018
6,2023-11-05 00:25:00,0.000000,1.337978,2.6,2,0.000000e+00,0.0,0.000000,0.0,0.668989,...,0.0,0.668989,0.0,1.0,23.238936,0.0,0.040848,0.063984,0.012035,0.668989
7,2023-11-05 00:30:00,0.000000,1.247968,2.6,2,8.313700e-03,0.0,0.000000,0.0,0.628141,...,0.0,0.628141,0.0,1.0,23.188456,0.0,0.000000,0.055670,0.012035,0.628141
8,2023-11-05 00:35:00,0.000000,1.256282,2.6,2,-1.110223e-16,0.0,0.000000,0.0,0.628141,...,0.0,0.628141,0.0,1.0,23.141058,0.0,0.023342,0.063984,0.012035,0.628141
9,2023-11-05 00:40:00,0.000000,1.209598,2.6,2,0.000000e+00,0.0,0.000000,0.0,0.604799,...,0.0,0.604799,0.0,1.0,23.093660,0.0,0.003104,0.000000,0.012035,0.604799


In [18]:
#dayAhead_sol_list.to_csv('dayAhead_sol_20231128_20231130.csv',index=False)
dayAhead_sol_list.to_csv('dayAhead_sol_test.csv',index=False)

In [19]:
#IntraDay_sol_list.to_csv('Intraday_sol_20231128_20231130.csv',index=False)
IntraDay_sol_list.to_csv('Intraday_sol_test.csv',index=False)

In [11]:
#dayAhead_sol_list = pd.read_csv('dayAhead_sol_NOV_2023_nolimit.csv',parse_dates=['datetime'])
#IntraDay_sol_list = pd.read_csv('Intraday_sol_NOV_2023_nolimit.csv',parse_dates=['datetime'])

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1,subplot_titles=('Solar generation','Load consumption'))

fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['PARAM_PV'],name='Predicted PV intraday'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['PARAM_PV'],name='Predicted PV dayAhead'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['PARAM_PL'],name='Predicted load intraday'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['PARAM_PL'],name='Predicted load dayAhead'),
    row=2, col=1
)
fig.update_yaxes(title_text = 'Power (kW)',row=1,col=1)
fig.update_yaxes(title_text = 'Power (kW)',row=2,col=1)
fig.update_layout(height=600, width=800, title_text="Predicted PV and Load")
fig.show()

In [13]:
intraday_sol[['uPnet','uPchg','uPdchg']].sum(axis=0)

uPnet     0.183638
uPchg     0.072210
uPdchg    8.137284
dtype: float64

In [14]:
dayAhead_sol_list[dayAhead_sol_list['datetime'] == '2023-11-01 23:45:00']

,datetime,PARAM_PV,PARAM_PL,Buy_rate,Sell_rate,Pnet,u1,s1,Pchg_0,Pdchg_0,...,soc_0,upper_bound_Pchg_0,upper_bound_Pdchg_0,Pchg_1,Pdchg_1,xchg_1,xdchg_1,soc_1,upper_bound_Pchg_1,upper_bound_Pdchg_1
95,2023-11-01 23:45:00,0.059217,2.934845,5.8,2,2.220446e-16,0.0,0.0,0.0,1.437814,...,56.516564,0.908422,1.437814,0.0,1.437814,0.0,1.0,56.516564,0.908422,1.437814


In [15]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=2,subplot_titles=('State of Charge (SoC)','Charge power','Expense with EMS 1','Discharge power'),specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}]])
excess_gen_DA = dayAhead_sol_list['PARAM_PV'] - dayAhead_sol_list['PARAM_PL']
excess_gen_HA = IntraDay_sol_list['PARAM_PV'] - IntraDay_sol_list['PARAM_PL']
cum_expense_DA = dayAhead_sol_list['u1'].cumsum()
cum_expense_HA = IntraDay_sol_list['u1'].cumsum()
# plot(1,1)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=excess_gen_HA, name = 'Excess gen HA',mode='lines+markers',line={'color':'#000000'}),
    row=1, col=1,secondary_y=True
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=excess_gen_DA,name = 'Excess gen DA',mode='lines+markers',line={'color':'#FF0000'}),
    row=1, col=1,secondary_y=True
)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['soc_0'], name = 'SoC HA',line={'color':'#000000'}),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['soc_0'],name = 'SoC DA',line={'color':'#FF0000'}),
    row=1, col=1
)

# plot (1,2)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['Pchg_0'], name = 'Pchg HA',line={'color':'#000000'}),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['Pchg_0'],name = 'Pchg DA',mode='markers',line={'color':'#FF0000'}),
    row=1, col=2
)
# plot (2,1)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=cum_expense_HA, name = 'Cumulative expense HA',line={'color':'#000000'}),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=cum_expense_DA,name = 'Cumulative expense DA',line={'color':'#FF0000'}),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['Pnet'], name = 'Pnet HA',mode='lines+markers',line={'color':'#000000'}),
    row=2, col=1, secondary_y=True
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['Pnet'],name = 'Pnet DA',mode='lines+markers',line={'color':'#FF0000'}),
    row=2, col=1, secondary_y=True
)

# plot (2,2)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['Pdchg_0'], name = 'Pdchg HA',line={'color':'#000000'}),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['Pdchg_0'],name = 'Pdchg DA',mode='markers',line={'color':'#FF0000'}),
    row=2, col=2
)

fig.update_yaxes(title_text='SoC (%)',row=1,col=1)
fig.update_yaxes(title_text='Excess gen (kW)',row=1,col=1,secondary_y=True)
fig.update_yaxes(title_text='Expense (THB)',row=2,col=1)
fig.update_yaxes(title_text='Pnet (kW)',row=2,col=1,secondary_y=True)
fig.update_yaxes(title_text='Power (kW)',row=1,col=2)
fig.update_yaxes(title_text='Power (kW)',row=2,col=2)
fig.update_layout(height=1000, width=1500, title_text="")
fig.show()